In [12]:
"""
# Spark CSV → Parquet Processing

## Goal
- GCS에 있는 CSV 데이터를 Spark로 읽는다
- 데이터 구조를 이해한다
- Parquet 포맷으로 변환한다
- 결과를 다시 GCS에 저장한다

## Pipeline
1. Spark Session 초기화
2. GCS 연동 설정
3. GCS CSV 로드
4. 데이터 탐색 (schema / sample)
5. Parquet 변환
6. GCS 업로드
"""

'\n# Spark CSV → Parquet Processing\n\n## Goal\n- GCS에 있는 CSV 데이터를 Spark로 읽는다\n- 데이터 구조를 이해한다\n- Parquet 포맷으로 변환한다\n- 결과를 다시 GCS에 저장한다\n\n## Pipeline\n1. Spark Session 초기화\n2. GCS 연동 설정\n3. GCS CSV 로드\n4. 데이터 탐색 (schema / sample)\n5. Parquet 변환\n6. GCS 업로드\n'

# Spark Session 초기화

In [12]:
import pyspark
pyspark.__version__

'3.5.8'

In [2]:
import os
from pyspark.sql import SparkSession

# 🔹 Hadoop 사용자 설정 (NullPointerException 방지)
os.environ["HADOOP_USER_NAME"] = "spark"

# 기존 SparkSession이 있으면 중지
try:
  spark.stop()
except:
  pass

spark = (
  SparkSession.builder
  .appName("csv_parquet")
  .master("spark://spark-master:7077")

  # 🔹 Ivy 캐시 경로 명시 (절대 경로)
  .config("spark.jars.ivy", "/home/spark/.ivy2")

  # 🔹 GCS FileSystem 연결 (gcs-connector 사용)
  .config(
      "spark.hadoop.fs.gs.impl",
      "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem"
  )
  .config(
      "spark.hadoop.fs.AbstractFileSystem.gs.impl",
      "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS"
  )

  # 🔹 GCS 인증 (서비스 계정)
  .config(
      "spark.hadoop.google.cloud.auth.service.account.enable", "true"
  )
  .config(
      "spark.hadoop.google.cloud.auth.service.account.json.keyfile",
      "/cred/clickstream-sa.json"
  )

  .getOrCreate()
)

# 로그 레벨 설정
spark.sparkContext.setLogLevel("WARN")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/28 12:03:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sparkContext.getConf().getAll()

[('spark.app.startTime', '1769513763042'),
 ('spark.hadoop.google.cloud.auth.service.account.json.keyfile',
  '/cred/clickstream-sa.json'),
 ('spark.driver.host', 'e1d772bd46f5'),
 ('spark.executor.id', 'driver'),
 ('spark.app.submitTime', '1769513762935'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --

In [3]:
!ls /home/jovyan/.config/gcloud

access_tokens.db		      config_sentinel  default_configs.db  logs
active_config			      configurations   gce
application_default_credentials.json  credentials.db   legacy_credentials


In [6]:
write_read(
    "clickstream-pipeline-484705-clickstream-data",
    "test/processed/hello.txt"
)


Hello world


# GCS CSV 로드

In [5]:
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "false")
    .csv(
        "gs://clickstream-pipeline-484705-clickstream-data/test/raw/2019-Nov.csv"
    )
)

df.printSchema()
df.show(5)


root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



[Stage 1:>                                                          (0 + 1) / 1]

+--------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|2019-11-01 00:00:...|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:...|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:...|      view|  17302664|2053013553853497655|                NULL| creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:...|      view|   3601530|2053013563810775923|appliances.kitche...|    lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:...|      view|   1004775|2053013555631882655|electronics.smart..

# 데이터 탐색 (schema / sample)

In [6]:
df.dtypes

[('event_time', 'string'),
 ('event_type', 'string'),
 ('product_id', 'string'),
 ('category_id', 'string'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'string'),
 ('user_id', 'string'),
 ('user_session', 'string')]

event_time   → timestamp<br>
price        → double <br>
product_id   → bigint  <br> 
category_id  → bigint   <br>
user_id      → bigint<br>

# data Type Casting

In [17]:
from pyspark.sql.functions import col, regexp_replace, to_timestamp

df_core = (
    df
    .withColumn(
        "event_time",
        to_timestamp(
            regexp_replace(col("event_time"), " UTC$", ""),
            "yyyy-MM-dd HH:mm:ss"
        )
    )
    .withColumn("price", col("price").cast("double"))
    .withColumn("product_id", col("product_id").cast("long"))
    .withColumn("category_id", col("category_id").cast("long"))
    .withColumn("user_id", col("user_id").cast("long"))
)


In [18]:
df_core.printSchema()
df_core.show(5)

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: long (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_session: string (nullable = true)

+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|53049679

# Parquet 변환

In [7]:
from pyspark.sql.functions import col, to_timestamp, to_date, date_format

df_parquet = (
  df
  .withColumn("event_time", to_timestamp(col("event_time"), "yyyy-MM-dd HH:mm:ss z"))
  .withColumn("event_date", to_date(col("event_time")))
  .withColumn("event_month", date_format(col("event_time"), "yyyy-MM"))
  .withColumn("product_id", col("product_id").cast("long"))
  .withColumn("category_id", col("category_id").cast("long"))
  .withColumn("price", col("price").cast("double"))
  .withColumn("user_id", col("user_id").cast("long"))
)

(
  df_parquet
  .write
  .mode("overwrite")
  .partitionBy("event_month", "event_date")
  .option("compression", "snappy")
  .parquet("/home/spark/work/output/parquet")
)

print(f"✅ Saved to: /home/spark/work/output/parquet")                                                                                                                        
print(f"   Host path: ./spark/output/parquet")  

[Stage 3:========================================================>(67 + 1) / 68]

✅ Saved to: /home/spark/work/output/parquet
   Host path: ./spark/output/parquet


In [ ]:
#셀확인
!ls -lh /home/spark/work/output/parquet 

In [ ]:
!du -sh /home/spark/work/output/parquet

# GCS 업로드

In [3]:
spark.sparkContext.uiWebUrl

'http://f40ced364137:4040'